In [66]:
#Data Analysis
import numpy as np
import os
import json
trainexamples = []
testexamples = []

def loadtest(direc):
    for each in os.listdir(direc):        
        if '.npy' in each:
            fids = each.split('_')[1][:-4]
            examples = np.load(direc+'/'+each)
            with open(direc+'/'+each[:-4]+'.json', 'r') as infile:
                values = json.load(infile)
            yield examples,fids,values
    return

   
def load(direc):
    examples = []
    fids  = []
    values = []
    for each in os.listdir(direc):        
        try:
            if '.npy' in each:
                fids.append(each.split('_')[1][:-4])
                examples.extend(np.load(direc+'/'+each))
                with open(direc+'/'+each[:-4]+'.json', 'r') as infile:
                    values.extend(json.load(infile))
        except:
            continue
    return examples,fids,values

trainexamples, trainfids, trainvalues = load('../../../data/Datefeature/cv1/train')

trainlabels, trainfeatures = [], []
for each in trainexamples:
    trainfeatures.append(each[1:])
    trainlabels.append(each[0])

testexamples, testfids, testvalues = load('../../../data/Datefeature/cv1/test')
testfeatures, testlabels = [], []
for each in testexamples:
    testfeatures.append(each[1:])
    testlabels.append(each[0])
assert(len(trainvalues)==len(trainlabels))
assert(len(testvalues)==len(testlabels))
print len(trainfeatures[0])


10


In [67]:
#expolaratory analysis
#does feature space capture 
print trainlabels.count(0),trainlabels.count(3)
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
pca = PCA(n_components=2)
#trainfeatures1 = normalize(trainfeatures)
X = pca.fit_transform(trainfeatures1)
assert(X.shape[1] == 2)

d = {0:[],1:[],2:[],3:[]}
for i,each in enumerate(trainexamples):
    d[each[0]].append(X[i,:])

x,y = zip(*d[0])
plt.plot(x,y,'bo')
x,y = zip(*d[3])
plt.plot(x,y,'ro')
plt.show()

5679 26


In [68]:
#Content extractor
#non parametric classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import numpy as np
# dc = DecisionTreeClassifier()
# trainfeatures1 = normalize(trainfeatures)
testlabels_date = map(lambda x: 1 if x==3 else 0 , testlabels)
trainlabels_date = map(lambda x: 1 if x==3 else 0 , trainlabels)
dc.fit(trainfeatures,trainlabels_date)
print f1_score(dc.predict(trainfeatures),trainlabels_date)
p_dectree = dc.predict(np.array(testfeatures))
print f1_score(p_dectree,testlabels_date)
print testlabels_date.count(1)
cnt = 0
print '---------Title---------'
for i,each in enumerate(zip(p_dectree,testlabels_date)):
    if each[0] == 0 and each[1]==1:
        print 'Mine %d Ground %d'%(each[0],each[1])
        print '-------------Error number %d------------' %cnt
        print testvalues[i]
        print testfeatures[i]
        cnt+=1
    if each[0] == 1 and each[1] == 1:
        print 'right'
    if each[0]==1:
        print '-----'
        print testvalues[i]
        print '-----'


0.790697674419
0.478260869565
22
---------Title---------
right
-----
December 29, 2010
-----
right
-----
 5:22 PM EDT Sep 17, 2015
-----
right
-----
June 22, 2015 5:51 PM
-----
Mine 0 Ground 1
-------------Error number 0------------
Tuesday, June 16, 2015 4:08 pm
[   1.  119.    1.    0.    1.    0.    0.    6.    0.    0.]
Mine 0 Ground 1
-------------Error number 1------------
Posted: 

[ 1.  1.  0.  0.  0.  0.  0.  2.  0.  0.]
right
-----
11:34 p.m. EST December 4, 2014
-----
-----
2015-06-11 06:02:55
-----
Mine 0 Ground 1
-------------Error number 2------------

            06/11/2015  6:15 pm EDT        
[  1.   2.   0.   0.   0.   0.   0.  25.   1.   0.]
Mine 0 Ground 1
-------------Error number 3------------
Updated: Mon 11:08 AM, Sep 14, 2015
[ 1.  3.  1.  0.  0.  0.  1.  7.  0.  0.]
-----
Monday, Sept. 14, 2015
-----
-----
Monday, Sept. 14, 2015
																				
-----
Mine 0 Ground 1
-------------Error number 4------------
Updated 10:04 PM ET, Mon April 14, 2014
[  1.  19.

In [ ]:
class TitleClassifier:
    
    def __init__(self):
        """Predicts for a single news article only"""
        pass
    
    def predict(self,X):        
        assert(X.shape[1] == 2)        
        rowno = 0
        res = -1
        gotit = False
        for goodmatch,isheading in X:
            
            if goodmatch == 1 and isheading==1:
                res = rowno
                
                break;
            if isheading == 1 and not gotit:
                res = rowno
                gotit  = True
            rowno+=1
        Y = [0]*X.shape[0]
        print res
        if res>=0:
            Y[res] = 1
        return Y
            
TestCheck = TitleClassifier()
import pickle
pickle.dump( TestCheck, file('Best_title.dat','wb'))
TC = pickle.load(file('Best_title.dat','rb'))
for x,y,v in loadtest('../../../../data/Titlefeature/cv1/test'):
    Y = TC.predict(np.array(x).transpose()[1:,].transpose())
    print '--**'*10
    title = ''
    for i,e in enumerate(Y):
        if e == 1:
            title+=v[i]
    print title
